In [ ]:
import heapq

class State:
    def __init__(self, n, source, target, intermediate, g_cost=0, h_cost=0, parent=None):
        self.n = n
        self.source = source
        self.target = target
        self.intermediate = intermediate
        self.g_cost = g_cost
        self.h_cost = h_cost
        self.parent = parent

    def __lt__(self, other):
        return (self.g_cost + self.h_cost) < (other.g_cost + other.h_cost) or self.g_cost < other.g_cost # if cost same then prefers on e with lesser g

def unique_hash(state):
    l = []
    l.extend(state.source)
    l.append(0)
    l.extend(state.intermediate)
    l.append(0)
    l.extend(state.target)
    return hash(tuple(l)) # converts list into tuple  as tuples are hashable

def h_cost(state):
    return state.n / state.target[0] if state.target else state.n # h based on number of pegs remaining to be moved to the target..remaining to be moved/smallest on target

def is_goal(state):
    return state.n == 0 # n of disk remaining to be moved in the current stae

def generate_successors(state):
    successors = []

    # Moving one disk from source to target
    if state.source and (not state.target or state.source[-1] < state.target[-1]):
        new_n = state.n - 1
        new_source = state.source[:-1]
        new_target = state.target.copy()
        new_target.append(state.source[-1])
        new_intermediate = state.intermediate.copy()
        successors.append(State(new_n, new_source, new_target, new_intermediate, state.g_cost + 1, h_cost(State(new_n, new_source, new_target, new_intermediate)), state))

    # Moving one disk from intermediate to target
    if state.intermediate and (not state.target or state.intermediate[-1] < state.target[-1]):
        new_n = state.n - 1
        new_source = state.source.copy()
        new_target = state.target.copy()
        new_target.append(state.intermediate[-1])
        new_intermediate = state.intermediate[:-1]
        successors.append(State(new_n, new_source, new_target, new_intermediate, state.g_cost + 1, h_cost(State(new_n, new_source, new_target, new_intermediate)), state))

    # Moving one disk from source to intermediate
    if state.source and (not state.intermediate or state.source[-1] < state.intermediate[-1]):
        new_n = state.n
        new_source = state.source[:-1]
        new_target = state.target.copy()
        new_intermediate = state.intermediate.copy()
        new_intermediate.append(state.source[-1])
        successors.append(State(new_n, new_source, new_target, new_intermediate, state.g_cost + 1, h_cost(State(new_n, new_source, new_target, new_intermediate)), state))

    # Moving one disk from target to intermediate
    if state.target and (not state.intermediate or state.target[-1] < state.intermediate[-1]):
        new_n = state.n + 1
        new_source = state.source.copy()
        new_intermediate = state.intermediate.copy()
        new_intermediate.append(state.target[-1])
        new_target = state.target[:-1]
        successors.append(State(new_n, new_source, new_target, new_intermediate, state.g_cost + 1, h_cost(State(new_n, new_source, new_target, new_intermediate)), state))

    # Moving one disk from intermediate to source
    if state.intermediate and (not state.source or state.intermediate[-1] < state.source[-1]):
        new_n = state.n
        new_source = state.source.copy()
        new_source.append(state.intermediate[-1])
        new_target = state.target.copy()
        new_intermediate = state.intermediate[:-1]
        successors.append(State(new_n, new_source, new_target, new_intermediate, state.g_cost + 1, h_cost(State(new_n, new_source, new_target, new_intermediate)), state))

    # Moving one disk from target to source
    if state.target and (not state.source or state.target[-1] < state.source[-1]):
        new_n = state.n + 1
        new_source = state.source.copy()
        new_source.append(state.target[-1])
        new_target = state.target[:-1]
        new_intermediate = state.intermediate.copy()
        successors.append(State(new_n, new_source, new_target, new_intermediate, state.g_cost + 1, h_cost(State(new_n, new_source, new_target, new_intermediate)), state))

    return successors

def reconstruct_path(state):
    path = []
    current_state = state
    while current_state is not None:
        path.append(current_state)
        current_state = current_state.parent
    return path[::-1] # tells the path that lead to solution

def ao_star_search(initial_state):
    open_set = []
    heapq.heappush(open_set, initial_state)
    closed_set = set()

    while open_set:
        current_state = heapq.heappop(open_set)

        if is_goal(current_state):
            return reconstruct_path(current_state)

        closed_set.add(unique_hash(current_state))

        successors = generate_successors(current_state)
        for successor in successors:
            if unique_hash(successor) in closed_set:
                continue
            heapq.heappush(open_set, successor)

    return None

def tower_of_hanoi(num_disks):
    source = list(range(num_disks, 0, -1))
    target = []
    intermediate = []
    initial_state = State(num_disks, source, target, intermediate)
    solution = ao_star_search(initial_state)
    if solution:
        print("Solution found with", len(solution) - 1, "moves:")
        for i, state in enumerate(solution):
            print("Move:", i, " Disks:", state.n, " Source:", state.source, " Target:", state.target, " Intermediate:", state.intermediate)
    else:
        print("No solution found.")

if __name__ == "__main__":
    num_disks = int(input("Enter the number of disks: "))
    tower_of_hanoi(num_disks)


Enter the number of disks: 4
Solution found with 15 moves:
Move: 0  Disks: 4  Source: [4, 3, 2, 1]  Target: []  Intermediate: []
Move: 1  Disks: 4  Source: [4, 3, 2]  Target: []  Intermediate: [1]
Move: 2  Disks: 3  Source: [4, 3]  Target: [2]  Intermediate: [1]
Move: 3  Disks: 2  Source: [4, 3]  Target: [2, 1]  Intermediate: []
Move: 4  Disks: 2  Source: [4]  Target: [2, 1]  Intermediate: [3]
Move: 5  Disks: 3  Source: [4, 1]  Target: [2]  Intermediate: [3]
Move: 6  Disks: 4  Source: [4, 1]  Target: []  Intermediate: [3, 2]
Move: 7  Disks: 4  Source: [4]  Target: []  Intermediate: [3, 2, 1]
Move: 8  Disks: 3  Source: []  Target: [4]  Intermediate: [3, 2, 1]
Move: 9  Disks: 2  Source: []  Target: [4, 1]  Intermediate: [3, 2]
Move: 10  Disks: 2  Source: [2]  Target: [4, 1]  Intermediate: [3]
Move: 11  Disks: 3  Source: [2, 1]  Target: [4]  Intermediate: [3]
Move: 12  Disks: 2  Source: [2, 1]  Target: [4, 3]  Intermediate: []
Move: 13  Disks: 2  Source: [2]  Target: [4, 3]  Intermediate:

In [ ]:
import numpy as np

# Define the original matrix
original_matrix = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3], [4, 4, 4]])

# Perform Singular Value Decomposition (SVD)
U, S, VT = np.linalg.svd(original_matrix)

# Choose the number of components (k) for factorization
k = 3

# Construct factor matrices
factor_matrix_U = U[:, :k]
factor_matrix_VT = np.diag(S[:k]).dot(VT[:k, :])

# Reconstruct the original matrix
reconstructed_matrix = np.dot(factor_matrix_U, factor_matrix_VT)

# Print the factor matrices
print("Factor Matrix U:")
print(factor_matrix_U)
print("\nFactor Matrix V^T:")
print(factor_matrix_VT)

# Print the reconstructed matrix
print("\nReconstructed Matrix:")
print(reconstructed_matrix)


Factor Matrix U:
[[-0.18257419  0.96896279  0.03333333]
 [-0.36514837 -0.1090576   0.9231634 ]
 [-0.54772256  0.04053774 -0.16713548]
 [-0.73029674 -0.2181152  -0.34456343]]

Factor Matrix V^T:
[[-5.47722558e+00 -5.47722558e+00 -5.47722558e+00]
 [ 7.25194643e-16 -3.62597321e-16 -3.62597321e-16]
 [ 0.00000000e+00  3.46194909e-47 -3.46194909e-47]]

Reconstructed Matrix:
[[1. 1. 1.]
 [2. 2. 2.]
 [3. 3. 3.]
 [4. 4. 4.]]
